In [3]:
import surprise
import pandas as pd
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate

In [20]:
analyzedata = pd.read_pickle("dataSet/analyzedata.pkl")

In [21]:
cl0 = analyzedata[analyzedata["cluster"] == 0]
cl1 = analyzedata[analyzedata["cluster"] == 1]
cl2 = analyzedata[analyzedata["cluster"] == 2]

In [43]:
cl2

,고객번호,성별,연령대,거주지역,A선호점포,B선호점포,C선호점포,D선호점포,선호제휴사,선호점포,...,최근성,행동 빈도,구매 총액,평균구매액,최대값1,최대값2,최대값3,LABEL,고객번호,cluster
1,00002,M,60세이상,100,7.0,18.0,NaN,33.0,A,7,...,2,1116,99880760,89498.89,일상,전문스포츠,디지털,1,00002,2.0
3,00004,F,60세이상,016,20.0,8.0,19.0,NaN,A,20,...,1,930,16693810,17950.33,기타,전문스포츠,신선,1,00004,2.0
13,00014,F,60세이상,043,1.0,58.0,15.0,NaN,A,1,...,1,1312,34567736,26347.36,기타,패션잡화,신선,1,00014,2.0
20,00021,M,60세이상,055,2.0,13.0,154.0,NaN,B,13,...,1,2194,22719564,10355.32,신선,가공,일상,1,00021,2.0
26,00027,M,60세이상,060,12.0,7.0,490.0,NaN,A,12,...,6,648,60061016,92686.75,일상,가공,신선,1,00027,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19311,19312,F,35세~39세,043,1.0,9.0,NaN,15.0,A,1,...,3,746,67117358,89969.65,일상,패션잡화,명품,1,19312,2.0
19323,19324,F,40세~44세,None,12.0,NaN,NaN,NaN,A,12,...,29,519,35767600,68916.38,패션잡화,기타,일상,1,19324,2.0
19329,19330,M,55세~59세,610,19.0,114.0,61.0,NaN,B,114,...,1,1274,12507304,9817.35,의약,신선,전문스포츠,1,19330,2.0
19333,19334,M,35세~39세,310,40.0,73.0,147.0,NaN,B,73,...,11,1964,12353531,6289.99,의약,신선,가공,1,19334,2.0


In [33]:
cl2_list=  list(cl2['고객번호'].iloc[:, 0])

In [149]:
df = pd.read_pickle("dataSet/purprod4.pkl")
df = df[df["고객번호"].isin(cl2_list)]

In [39]:
df1 = pd.read_pickle("dataSet/purprod2.pkl")
df1

,소분류코드,임의코드,소분류명,임의분류명,구매빈도
0,A010301,A01,햄,가공식품,4
1,A010302,A01,유제품,가공식품,40
2,A010304,A01,화과자,가공식품,1
3,A010505,A01,냉동식품,가공식품,4
4,A010509,A01,안주류,가공식품,1
...,...,...,...,...,...
6527915,D080205,D03,메이크업세트,일상용품,1
6527916,D080302,D03,립글로즈/틴트,일상용품,2
6527917,D080401,D03,일반네일/케어류,일상용품,1
6527918,D020501,D05,생리대,의약품/의료기기,3


In [150]:
reader = surprise.Reader(rating_scale = (1, 5))

data  = Dataset.load_from_df(df[['고객번호', '소분류코드', 'RANK1']], reader)

trainset, testset = train_test_split(data, test_size = .25, random_state = 0)

algo = SVD(random_state = 0)
algo.fit(trainset)

cross_validate(algo, data, measures = ['RMSE', 'MAE'], cv = 5, verbose = True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5379  1.5345  1.5351  1.5310  1.5314  1.5340  0.0025  
MAE (testset)     1.2663  1.2651  1.2664  1.2614  1.2615  1.2641  0.0023  
Fit time          3.78    3.80    3.85    3.78    3.85    3.81    0.03    
Test time         1.01    1.02    0.79    1.00    1.00    0.96    0.09    


{'test_rmse': array([1.53787242, 1.53448542, 1.53509902, 1.53096507, 1.53142524]),
 'test_mae': array([1.26630363, 1.2650568 , 1.2664232 , 1.26137273, 1.26149736]),
 'fit_time': (3.78383469581604,
  3.795827627182007,
  3.854794502258301,
  3.784839391708374,
  3.846799850463867),
 'test_time': (1.009422779083252,
  1.018418788909912,
  0.7875502109527588,
  1.0004231929779053,
  0.9964292049407959)}

In [66]:
# from surprise.model_selection import GridSearchCV

# param_grid = {'n_epochs' : [20, 40, 60, 80, 100], 'n_factors' :  [50, 100, 200]}

# gs = GridSearchCV(SVD, param_grid, measures = ['rmse', 'mae'], cv = 3)
# gs.fit(data)

# print(gs.best_score['rmse'])
# print(gs.best_params['rmse'])

1.5445235033355071
{'n_epochs': 20, 'n_factors': 50}


In [15]:
# algo = SVD(n_epochs = 100, n_factors = 100, random_state = 0)
# algo.fit(trainset)

In [53]:
def get_unpur_surprise(df, df1, 고객번호) :
    
    purs = df[df['고객번호'] == 고객번호]['소분류코드'].tolist()
    
    total_pur = df['소분류코드'].tolist()
    
    unpurs = [pur for pur in total_pur if pur not in purs]
    
    print('평점 매긴 제품수 : ', len(purs), '추천대상 제품수 : ', len(unpurs),
         '전체 제품수 : ', len(total_pur))
    
    return unpurs

unpurs = get_unpur_surprise(df, df, '00021')

평점 매긴 제품수 :  491 추천대상 제품수 :  313960 전체 제품수 :  450628


In [189]:
def get_unpur_surprise(df, df1, 고객번호) :
    
    purs = df[df['고객번호'] == 고객번호]['소분류코드'].tolist()
    
    total_pur = df['소분류코드'].tolist()
    
    unpurs = [pur for pur in total_pur if pur not in purs]
    
    print('평점 매긴 제품수 : ', len(purs), '추천대상 제품수 : ', len(unpurs),
         '전체 제품수 : ', len(total_pur))
    
    return unpurs


def recomm_pur_by_surprise(algo, 고객번호, unpurs, top_n = 5) :
    
    preds = []
    predicted_items = set()  # 중복 호출 방지를 위한 셋(set)

    for 소분류코드 in unpurs:
        if 소분류코드 not in predicted_items:  # 이미 예측한 소분류코드인지 확인
            pred = algo.predict(str(고객번호), str(소분류코드))  # 예측 객체 생성
            preds.append(pred)  # 예측 객체를 리스트에 추가
            predicted_items.add(소분류코드)  # 예측한 소분류코드를 셋에 추가
    
    
    def sortkey_est(pred) :
        
        return pred.est
    
    preds.sort(key = sortkey_est, reverse = True)
    top_predictions = preds[:top_n]
    
    top_pur_ids = [str(pred.iid) for pred in top_predictions]
    top_pur_rating = [pred.est for pred in top_predictions]
    top_pur_titles = df1[df1.소분류코드.isin(top_pur_ids)]['소분류명']
    top_pur_preds = [(id, title, rating) for id, title, rating in zip(top_pur_ids, top_pur_titles, top_pur_rating)]
    
    return top_pur_preds

unpurs = get_unpur_surprise(df, df, '11680')
top_pur_preds = recomm_pur_by_surprise(algo, '11680', unpurs, top_n = 5)
print('#### Top-5 추천 제품 리스트 ####')

for top_pur in top_pur_preds:
    print(top_pur[1], ':', top_pur[2])

평점 매긴 제품수 :  187 추천대상 제품수 :  363205 전체 제품수 :  450628
#### Top-5 추천 제품 리스트 ####
기초A : 4.284871267672991
엘레강스 : 4.1838091486921565
건강식품(비타민) : 4.006613473480854
기초A : 3.9082595970563188
엘레강스 : 3.8959743127976187


In [83]:
cl2.sort_values("구매 총액", ascending=False)["고객번호"][:50]

,고객번호,고객번호
8078,08079,08079
96,00097,00097
11762,11763,11763
2289,02290,02290
946,00947,00947
9517,09518,09518
561,00562,00562
9405,09406,09406
8101,08102,08102
9775,09776,09776
